In [3]:
# stock_data_fetcher.py
import pandas as pd
from datetime import date 

import requests
import datetime
from sqlalchemy import  Column, Integer, String, Float, Date
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import sessionmaker
from contextlib import contextmanager
from typing import Optional

# from stock_data_fetcher import StockDataFetcher
from get_api_key import get_api_key 
from sqlmodel import (
    Field,
    Session,
    SQLModel,
    create_engine,
    select
)
# Define database model
# Base = declarative_base()

_db_engine = None

class StockPrice(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    symbol: str = Field(index=True, max_length=10)
    stock_date: date = Field(index=True)
    open: float
    high: float
    low: float
    close: float
    volume: int

# Database connection
def get_db_engine(db_url="sqlite:///stocks.db"):
    global _db_engine
    if _db_engine is None:
        _db_engine = create_engine(db_url)
    return _db_engine

def init_db(engine):
    SQLModel.metadata.drop_all(engine)
    SQLModel.metadata.create_all(engine)
    return Session(engine)

@contextmanager
def db_engine_context(db_url="sqlite:///stocks.db"):
    global _db_engine
    try:
        engine = get_db_engine(db_url)
        yield engine
    finally:
        if _db_engine is not None:
            _db_engine.dispose()
            _db_engine = None

# Alpha Vantage API for stock data
class StockDataFetcher:
    def __init__(self, api_key: str, db_session: Session):
        self.api_key = api_key
        self.base_url = "https://www.alphavantage.co/query"
        self.db_session = db_session

    def fetch_daily_data(self, symbol: str, full: bool = True):
        """
        Fetch daily stock data for a given symbol using Alpha Vantage API, returning a DataFrame.
        """
        # Now using SQLModel queries via session.exec(select(...)).
        latest_entry = self.db_session.exec(
            select(StockPrice)
            .where(StockPrice.symbol == symbol)
            .order_by(StockPrice.stock_date.desc())
        ).first()

        if latest_entry and latest_entry.stock_date == datetime.date.today():
            print(f"Data for {symbol} is already up-to-date.")
            return "up_to_date"

        output_size = "full" if full else "compact"
        params = {
            "function": "TIME_SERIES_DAILY",
            "symbol": symbol,
            "outputsize": output_size,
            "apikey": self.api_key
        }

        response = requests.get(self.base_url, params=params)
        data = response.json()

        if "Time Series (Daily)" not in data:
            print(f"Error fetching data for {symbol}: {data.get('Information', 'Unknown error')}")
            return None

        time_series = data["Time Series (Daily)"]
        df = pd.DataFrame.from_dict(time_series, orient='index')
        df = df.rename(
            columns={
                '1. open': 'open',
                '2. high': 'high',
                '3. low': 'low',
                '4. close': 'close',
                '5. volume': 'volume'
            }
        )

        # Convert data types
        for col in ['open', 'high', 'low', 'close']:
            df[col] = df[col].astype(float)
        df['volume'] = df['volume'].astype(int)

        df['symbol'] = symbol
        df.index = pd.to_datetime(df.index)
        df = df.reset_index().rename(columns={'index': 'date'})
        return df

    def save_to_db(self, df: pd.DataFrame):
        """
        Save the incoming DataFrame to the database via SQLModel.
        """
        if df is None or df.empty:
            return False

        for _, row in df.iterrows():
            existing = self.db_session.exec(
                select(StockPrice).where(
                    StockPrice.symbol == row["symbol"],
                    StockPrice.stock_date == row["date"]
                )
            ).first()

            if not existing:
                stock_price = StockPrice(
                    symbol=row['symbol'],
                    stock_date=row['date'],
                    open=row['open'],
                    high=row['high'],
                    low=row['low'],
                    close=row['close'],
                    volume=row['volume']
                )
                self.db_session.add(stock_price)

        self.db_session.commit()
        return True

    def update_stocks(self, symbols):
        """
        Update stock data for a list of symbols. By default, fetches recent data (compact).
        """
        results = {}
        for symbol in symbols:
            print(f"Fetching data for {symbol}...")
            response = self.fetch_daily_data(symbol, full=False)

            if response == "up_to_date":
                results[symbol] = True
            elif response is None:
                results[symbol] = False
            else:
                success = self.save_to_db(response)
                results[symbol] = success
        return results

    def get_stock_data(self, symbol: str, days: int = 30):
        """
        Get in-database stock data for a given symbol over N days, returning a list of dicts.
        """
        cutoff_date = datetime.datetime.now() - datetime.timedelta(days=days)
        query = (
            select(StockPrice)
            .where(StockPrice.symbol == symbol)
            .where(StockPrice.stock_date >= cutoff_date)
            .order_by(StockPrice.stock_date)
        )
        stock_prices = self.db_session.exec(query).all()
        print(stock_prices)

        return [
            {
                "symbol": row.symbol,
                "date": row.stock_date.isoformat(),
                "open": row.open,
                "high": row.high,
                "low": row.low,
                "close": row.close,
                "volume": row.volume,
            }
            for row in stock_prices
        ]

    def reinitialize_db(self, engine, symbols=None):
        """
        Drops all tables (if they exist) and recreates them via SQLModel, 
        then fetches "full" daily stock data for the specified symbols.
        """
        if symbols is None:
            symbols = ["AAPL", "MSFT", "AMZN"]  # Default symbols

        # 1) Drop any existing tables
        print("Dropping existing tables...")
        SQLModel.metadata.drop_all(engine)  # <--- SQLModel drop_all
        
        # 2) Recreate tables
        print("Creating fresh tables...")
        SQLModel.metadata.create_all(engine)

        # 3) Fetch full (historical) data for each symbol
        with Session(engine) as session:
            fetcher = StockDataFetcher(api_key=self.api_key, db_session=session)
            for symbol in symbols:
                df = fetcher.fetch_daily_data(symbol, full=True)
                if df is not None and df != "up_to_date":
                    fetcher.save_to_db(df)
            print(f"Fetched fresh data for symbols: {symbols}")

In [8]:
API_KEY = get_api_key("ALPHA_VANTAGE_API_KEY")  # Replace with your API key
    
with db_engine_context() as engine:
    session = init_db(engine)

    
    
    fetcher = StockDataFetcher(API_KEY, session)
    # Update data for specified stocks
    stocks = ["AAPL", "MSFT", "GOOG", "AMZN"]
    # results = fetcher.update_stocks(stocks)

    results = fetcher.get_stock_data("AAPL")
    # for stock in stocks:
    #     results = fetcher.get_num_rows("AAPL")

    #     print("Update results:", results)

[]


In [5]:
print(results)

[]


In [7]:
API_KEY = get_api_key("ALPHA_VANTAGE_API_KEY")  # Replace with your API key
symbols = ["AAPL", "MSFT", "AMZN","GOOG"]
with db_engine_context() as engine:
    print("Dropping existing tables...")
    # SQLModel.metadata.drop_all(engine)

    print("Creating fresh tables...")
    SQLModel.metadata.create_all(engine)

    with Session(engine) as session:
        fetcher = StockDataFetcher(api_key=API_KEY, db_session=session)
        for symbol in symbols:
            df = fetcher.fetch_daily_data(symbol, full=True)
            # if df is not None and df != "up_to_date":
            fetcher.save_to_db(df)
        print(f"Fetched fresh data for symbols: {symbols}")


Dropping existing tables...
Creating fresh tables...
Error fetching data for AAPL: Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.
Error fetching data for MSFT: Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.
Error fetching data for AMZN: Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.
Error fetching data for GOOG: Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instant